In [1]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# connect_to_apiメソッド

```python
def connect_to_api(module, disconnect_atexit=True)
```

[ServiceInstance](https://vdc-download.vmware.com/vmwb-repository/dcr-public/6b586ed2-655c-49d9-9029-bc416323cb22/fa0b429a-a695-4c11-b7d2-2cbc284049dc/doc/vim.ServiceInstance.html) から [ServiceContent](https://vdc-download.vmware.com/vmwb-repository/dcr-public/6b586ed2-655c-49d9-9029-bc416323cb22/fa0b429a-a695-4c11-b7d2-2cbc284049dc/doc/vim.ServiceInstanceContent.html) Objectを取得するメソッドです。  
vCenterまたはESXiへ接続する時に使用するメソッドです。

## 引数

|        args       |  type |                  param                   |
|-------------------|-------|------------------------------------------|
| module            | class | ansible.module_utils.basic.AnsibleModule |
| disconnect_atexit | book  | 処理終了後に接続を自動で切断するフラグ   |

## 戻り値

| rtype |                      return                     |
|-------|-------------------------------------------------|
| class | pyVmomi.VmomiSupport.vim.ServiceInstanceContent |

# connect_to_apiメソッド動作確認

connect_to_apiメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* vm_name
* hostname
* username
* password
* validate_certs

ServiceContent Objectが取得できることを確認します。

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_obj
from pyVmomi import vim

vm_name = 'devel'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
print(sc)

# connect_to_apiメソッドを使ったモジュール例

connect_to_apiを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはServiceContentを取得するだけの単純なものです。  
モジュールは `/tmp/ansible_salf_made_module` に `vmware_connect.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/vmware_connect.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    result = dict(changed=False)
    sc = connect_to_api(module)
    if(sc):
        result["sc"] = sc
        module.exit_json(**result)
    else:
        module.fail_json(msg="connect fail.")

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
vmware_connect:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no

`ServiceContent` が取得できていることを確認します。